In [1]:
from espnet2.bin.asr_inference_streaming import Speech2TextStreaming
from espnet_model_zoo.downloader import ModelDownloader
import numpy as np
import pyaudio
import torch

Failed to import Flash Attention, using ESPnet default: No module named 'flash_attn'


C:\Users\Dell\PycharmProjects\Realtime_ASR\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# VAD model setup
model, utils = torch.hub.load('snakers4/silero-vad', 'silero_vad', force_reload=True, skip_validation=True)
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils
vad_iterator = VADIterator(model, threshold=0.5, sampling_rate=16000)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\Dell/.cache\torch\hub\master.zip


In [5]:
tag        = "D-Keqi/espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave"
d          = ModelDownloader()
model_info = d.download_and_unpack(tag)

# Initialize streaming ASR
speech2text = Speech2TextStreaming(
    **model_info,
    token_type="bpe",
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=5,          # Faster streaming
    ctc_weight=0.7,       # Hybrid CTC/attention
    lm_weight=0.3,        # No external LM
    penalty=0.0,
    nbest=1,
    device="cpu",        # Use GPU if available
    disable_repetition_detection=True,  # Avoid stuck loops
)

print('recording ........')

CHUNK    = 512 # 2048 5120
FORMAT   = pyaudio.paInt16
CHANNELS = 1
RATE     = 16000 # 16000

THRESHOLD = 100
counter = 0
flag = False

# Time per chunk
# ms = (chunk/rate) * 1000

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

CHUNKS_PER_SECOND = int(RATE/CHUNK)

while True:

    data = stream.read(CHUNK)

    data = np.frombuffer(data, dtype='int16')

    data_for_vad = data.astype(np.float32)/32767.0

    data = data.astype(np.float16)/32767.0

    is_speech = vad_iterator(data_for_vad, return_seconds=False)
    if not is_speech:
        counter += 1
    else:
        counter = 0

    if counter > THRESHOLD:
        flag = True
    else:
        flag = False

    try:
        results = speech2text(speech=data, is_final=flag)

        if flag:
            flag = False
            speech2text.reset()

        if results and results[0][0]:
            print(results[0][0])

    except RuntimeError as e:
        flag = False
        counter = 0

Fetching 32 files: 100%|██████████| 32/32 [00:00<00:00, 31797.61it/s]


recording ........
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
he
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hello my name is jon
hell

KeyboardInterrupt: 